In [6]:
# script to rescale, transform to real FA and average B1 dream maps
# save final img in my home folder, to later on move to SB-ref folder
from nilearn import image, plotting
%matplotlib inline
import os
import glob
import numpy as np
import nibabel as nib

In [7]:
# dir for new acquisitions of anatomies
sub_num = '12'
SB_dir_B1 = '/home/neuro/projects/data/sub-'+str(sub_num).zfill(2)+'/ses-anat/fmap/sub-'+str(sub_num).zfill(2)+'_ses-anat_acq-dream'

In [8]:
FA = ['30','45','60']
thresh = '50' #degree of FA to use for ceiling

for i in range(len(FA)):
    B1 = glob.glob(SB_dir_B1+FA[i]+'_B1map.nii.gz'); B1=B1[0] #path to B1 map
    check_img = nib.load(B1)
    check_data = check_img.get_data()
    max_int = np.amax(check_data) #check max intensity of image

    if max_int < 5: #IN THIS CASE CHECK DATA
        print('%s already rescaled, max intensity is %f' %(os.path.basename(B1).split(os.extsep)[0],max_int))
    else:      
        data = image.load_img(B1); print(data.shape) # import image
        data_b1 = image.index_img(data,0); print(data_b1.shape) # only 1st volume is B1 map (what are others?)
        new_b1 = image.math_img("img1/1000",img1=data_b1) # divide by 1000 to get a rescaled B1 (median intensity 1)
        new_b1.to_filename(os.path.basename(B1).split(os.extsep)[0]+'_rescaled.nii.gz') # save only image name, no extensions
        
        check_img = nib.load(os.path.basename(B1).split(os.extsep)[0]+'_rescaled.nii.gz')
        max_int = np.amax(check_img.get_data()) #new max intensity after rescaling 
        
        if max_int*float(FA[i]) < 50: #if it doesn't reach the tresh, only save nifti in my dir
            print('No ceiling because max intensity x %fº is %f for %s, lower than 50º' %(float(FA[i]),max_int*float(FA[i]),os.path.basename(B1).split(os.extsep)[0]+'_rescaled.nii.gz'))
            nib.save(new_b1, os.path.basename(B1).split(os.extsep)[0]+'_trueFA_50degthresh.nii.gz')
        else:       
            trFA_b1 = image.math_img('np.clip(img1*'+FA[i]+',0,'+thresh+')/'+FA[i],img1=new_b1) # get true FA img, ceilinged at 50º        
            trFA_b1.to_filename(os.path.basename(B1).split(os.extsep)[0]+'_trueFA_50degthresh.nii.gz') # save only image name, no extensions   
                     

(64, 64, 64, 6)
(64, 64, 64)
No ceiling because max intensity x 30.000000º is 26.802123 for sub-12_ses-anat_acq-dream30_B1map_rescaled.nii.gz, lower than 50º
(64, 64, 64, 6)
(64, 64, 64)
(64, 64, 64, 6)
(64, 64, 64)


In [9]:
#need to average
data1 = image.load_img('sub-'+str(sub_num).zfill(2)+'_ses-anat_acq-dream'+FA[0]+'_B1map_trueFA_50degthresh.nii.gz')
data2 = image.load_img('sub-'+str(sub_num).zfill(2)+'_ses-anat_acq-dream'+FA[1]+'_B1map_trueFA_50degthresh.nii.gz')
data3 = image.load_img('sub-'+str(sub_num).zfill(2)+'_ses-anat_acq-dream'+FA[2]+'_B1map_trueFA_50degthresh.nii.gz')
# concatenate all b1 maps and get average img
trFA_b1_avg = image.mean_img(image.concat_imgs([data1,data2,data3]))
trFA_b1_avg.to_filename('sub-'+str(sub_num).zfill(2)+'_ses-anat_acq-dreamAVG_B1map_trueFA_50degthresh.nii.gz') # save only image name, no extensions
